# Updating the Document Memory (on the fly)

To be able to learn our Agent system needs to be able to modify its memory on the fly. This feature particularly important in any learning system is often not present in Agent systems. Because actually integrating new documents or facts into memory include also the data processing pipeline needed to chunk/extract and embed the information.

With HybridAGI, we have a very elegant way of doing it by using on the fly data processing pipelines. Allowing the Agent to learn on the fly new knowledge. However this technique should not replace the traditional way of integrating prior knowledge to the system like presented in other notebooks, that is two different kind of applications. This way of populating the memory should be only used when working with an Agent system that needs to learn, or if you are building a scrapper Agent.

In [ ]:
import hybridagi.core.graph_program as gp

main = gp.GraphProgram(
    name = "main",
    description = "The main program",
)

main.add(gp.Action(
    id = "search_docs",
    purpose = "Search for relevant documents to answer the Objective's question",
    tool = "DocumentSearch",
    prompt = "Please use the Objective's question to infer the search query",
))

main.add(gp.Decision(
    id = "check_context",
    purpose = "Check if the answer to the Objective's question is in your Context",
    question = "Is the answer to the Objective's question in your context?"
))

main.add(gp.Action(
    id = "answer_context_based",
    purpose = "Answer to the Objective's question based on your Context",
    tool = "Speak",
    prompt = "Answer to the Objective's question, if there is relevant information in your Context, please use it",
))

main.add(gp.Action(
    id = "answer",
    purpose = "Answer to the Objective's question",
    tool = "Speak",
    prompt = "Answer to the Objective's question, don't say it is based on your search just answer",
))

main.add(gp.Action(
    id = "save_answer",
    purpose = "Save the answer to the Objective question into memory",
    tool = "AddDocument",
    prompt = "Use the answer in your context to infer the document to save, never explain what you are doing",
))

main.connect("start", "search_docs")
main.connect("search_docs", "check_context")
main.connect("check_context", "answer_context_based", label="Yes")
main.connect("check_context", "answer", label="No")
main.connect("answer", "save_answer")
main.connect("answer_context_based", "end")
main.connect("save_answer", "end")

main.build()

In [ ]:
# We add the programs into memory
from hybridagi.memory.integration.falkordb import FalkorDBProgramMemory, FalkorDBDocumentMemory
from hybridagi.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(
    model_name_or_path = "all-MiniLM-L6-v2",
    dim = 384,
)

program_memory = FalkorDBProgramMemory(
    index_name="update_memory",
    embeddings=embeddings,
    wipe_on_start=True,
)

program_memory.update(main)

# Then we instanciate the document memory for later use
document_memory = FalkorDBDocumentMemory(
    index_name="update_memory",
    embeddings=embeddings,
    wipe_on_start=True,
)


In [8]:
# Now we setup the agent and tools
import dspy
from hybridagi.core.datatypes import AgentState
from hybridagi.core.pipeline import Pipeline
from hybridagi.embeddings import SentenceTransformerEmbeddings
from hybridagi.modules.agents import GraphInterpreterAgent
from hybridagi.modules.splitters import DocumentSentenceSplitter
from hybridagi.modules.embedders import DocumentEmbedder
from hybridagi.modules.retrievers.integration.local import FAISSDocumentRetriever 
from hybridagi.modules.agents.tools import (
    SpeakTool,
    DocumentSearchTool,
    AddDocumentTool,
)

embeddings = SentenceTransformerEmbeddings(
    model_name_or_path = "all-MiniLM-L6-v2",
    dim = 384,
)

pipeline = Pipeline()

pipeline.add("chunk_documents", DocumentSentenceSplitter(
    method = "word",
    chunk_size = 1024,
    chunk_overlap = 0,
))
pipeline.add("embed_chunks", DocumentEmbedder(embeddings=embeddings))

agent_state = AgentState()

tools = [
    SpeakTool(
        agent_state = agent_state,
    ),
    DocumentSearchTool(
        retriever = FAISSDocumentRetriever(
            document_memory = document_memory,
            embeddings = embeddings,
            distance = "cosine",
            max_distance = 1.0,
            k = 5,
            reranker = None,
        ),
    ),
    AddDocumentTool(
        document_memory = document_memory,
        pipeline = pipeline # Here we bind the document processing pipeline to our tool
    ),
]

lm = dspy.OllamaLocal(model='mistral', max_tokens=1024, stop=["\n\n\n"])
dspy.configure(lm=lm)

agent = GraphInterpreterAgent(
    program_memory = program_memory,
    agent_state = agent_state,
    tools = tools,
)


In [ ]:
from hybridagi.core.datatypes import Query

# Now we can test our system
result = agent(Query(query="What the definition of a neuro symbolic AI"))

print(result.final_answer)

In [ ]:
# Ok so now we can actually try to run the agent again to see if the memory got updated correctly

result = agent(Query(query="What the definition of a neuro symbolic AI"))

print(result.final_answer)